In [ ]:
%%javascript
$('#header').toggle();

In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
#from win32api import GetSystemMetrics

#print("Width =", GetSystemMetrics(0))
#print("Height =", GetSystemMetrics(1))

Width = 1707
Height = 960


In [8]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Heatmap, Circle
from ipywidgets import HTML, Layout, Dropdown, Text, Button, VBox, Box, Label 
from ipywidgets import AppLayout, GridspecLayout, interact
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
from datetime import datetime

geolocator = Nominatim(user_agent="appMAP_cernicchiaro.giulia@gmail.com")

maps = {'Standard' : basemaps.OpenStreetMap.Mapnik,
        'Esri' : basemaps.Esri.DeLorme}

state = sorted(list(pd.read_csv('C:\\deconMAP\\sql-pays.csv', sep=',', low_memory=False, header=None)[4]))
#state = sorted(list(pd.read_csv('sql-pays.csv', sep=',', low_memory=False, header=None)[4]))

#deps = {x['NOM']:x['DEP'] for i,x in pd.read_csv('C:\\deconMAP\\DepFra.csv', sep=',', low_memory=False).iterrows()}


header = HTML("<h1>Find Destinations Around Me</h1>", layout=Layout(height='auto'))
header.style.text_align='center'

basemap_selector = Dropdown( options = list(maps.keys()), layout=Layout(width='auto') )
state_selector = Dropdown( options = list(state), layout=Layout(width='auto') )
#department_selector = Dropdown( options = list(deps.keys()), layout=Layout(width='auto') )

basemap_selector.value = 'Standard'
state_selector.value = 'France'
#department_selector.value = None

main_map = Map(basemap=basemaps.OpenStreetMap.Mapnik)
main_map.layout.height = 'auto'

center_map = geolocator.geocode(state_selector.value)
main_map.center = (center_map.latitude, center_map.longitude)
main_map.zoom = 6

circle = Circle()

user_selection = Text(placeholder='Saisir une adresse')
user_validation = Button(description='Valider', button_style='info', tooltip='Calculer le rayon')

user_selection.layout.width = '80%'
#user_selection.layout.height = 'auto'

out = HTML(
    value='',
    layout=Layout(width='auto', height='auto')
)


rows = []

def add_log(msg):
    max_rows = 100
    rows.append(msg)
    if len(rows) > max_rows:
        rows.pop(0)
    return '<h4>Activity log</h4><ul>{}</ul>'.format('<li>'.join([''] + rows))

def get_addresses(dep):
    file = 'C:\\deconMAP\\adresses_treated\\ban-' + str(dep) + '.csv.gz'
    return pd.read_csv(file, compression='gzip', sep=',', low_memory=False)

def handle_interaction(**kwargs):
    if kwargs['type'] == 'click':
        found_address = geolocator.reverse(str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]), language='fr')
        msg = '( %s ) Selected coordinates: lat %s , lon %s\n' % (datetime.now(), found_address.latitude, found_address.longitude)
        state_selector.value = found_address.address.split(', ')[-1]
        if state_selector.value == 'France':
            user_selection.value = found_address.address
        else:
            user_selection.value = found_address.address
        msg += '-> %s\n' % (state_selector.value)
        out.value = add_log(msg)
        circle.location = (kwargs['coordinates'][0], kwargs['coordinates'][1])
        circle.radius = 1
        circle.color = "blue"
        main_map.layers = [basemap_to_tiles(maps[basemap_selector.value]), circle]

def on_map_selected(change):
    main_map.layers = [basemap_to_tiles(maps[basemap_selector.value]), circle]

def on_state_selected(change):
    center_map = geolocator.geocode(state_selector.value)
    main_map.center = (center_map.latitude, center_map.longitude)
    main_map.zoom = 6

def on_button_clicked(b):
    #main_map.fireEvent('click',{latlng:[x,y]})
    found_address = geolocator.geocode(user_selection.value, language='fr')
    msg = '( %s ) Result : %s (%s , %s)\n' % (datetime.now(), found_address.address, found_address.latitude, found_address.longitude)
    state_selector.value = found_address.address.split(', ')[-1]
    out.value = add_log(msg)
    circle.location = (found_address.latitude, found_address.longitude)
    circle.radius = 1
    circle.color = "blue"
    main_map.layers = [basemap_to_tiles(maps[basemap_selector.value]), circle]
    circle_radius = Circle()
    circle_radius.location = (found_address.latitude, found_address.longitude)
    circle_radius.radius = 100000
    circle_radius.color = "red"
    circle_radius.fill_color = "green"
    main_map.add_layer(circle_radius)

basemap_selector.observe(on_map_selected, names='value')

state_selector.observe(on_state_selected, names='value')

main_map.on_interaction(handle_interaction)

user_validation.on_click(on_button_clicked)    


grid_header = GridspecLayout(1, 4, height='auto')
grid_header[0, :2] = header
grid_header[0, 2] = VBox([Label("Pays:"),state_selector])
grid_header[0, 3] = VBox([Label("Basemap:"),basemap_selector])

grid_footer = GridspecLayout(4, 1, height='auto',width='auto')
grid_footer[0,0] = VBox([Label("Adresse choisie:"),Box([user_selection,user_validation])])
grid_footer[1:2,0] = Box()
grid_footer[3,0] = out

AppLayout(header=grid_header,
          center=main_map,
          footer=grid_footer,
          #pane_widths=[0, 1, 0],
          pane_heights=['10%', '60%', '40%'],
          #width='1000px',
          height='1000px',
          grid_gap="5px")


AppLayout(children=(GridspecLayout(children=(HTML(value='<h1>Find Destinations Around Me</h1>', layout=Layout(…